In [ ]:
!pip install torch torchvision opencv-python pytesseract transformers
!pip install tesseract
!pip install pytesseract
!sudo apt-get install tesseract-ocr
!pip install ultralytics

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 10 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (11.5 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/argman/EAST.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q $git_repo_url
  #!cd $project_name && pip install -q -r requirements.txt
  # patch for the Python 3.7
  !sed -i 's/python3-config/python3.7-config/g' EAST/lanms/Makefile

import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

In [ ]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  #confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name


pretrained_model = 'east_icdar2015_resnet_v1_50_rbox'
if not exists(pretrained_model):
  # download the pretrained model
  pretrained_model_file_name = 'east_icdar2015_resnet_v1_50_rbox.zip'
  download_from_google_drive('0B3APw5BZJ67ETHNPaU9xUkVoV0U', pretrained_model_file_name)
  !unzip $pretrained_model_file_name

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  346M  100  346M    0     0  37.4M      0  0:00:09  0:00:09 --:--:-- 23.4M
Archive:  east_icdar2015_resnet_v1_50_rbox.zip
  inflating: east_icdar2015_resnet_v1_50_rbox/checkpoint  
  inflating: east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491.data-00000-of-00001  
  inflating: east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491.index  
  inflating: east_icdar2015_resnet_v1_50_rbox/model.ckpt-49491.meta  


In [ ]:
import torch
import cv2
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pytesseract
from imutils.object_detection import non_max_suppression
import numpy as np
from ultralytics import YOLO
from pathlib import Path
from PIL import Image
import pandas as pd

def decode_predictions(scores, geometry, min_confidence):
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
    for y in range(0, numRows):
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        # Loop over the number of columns
        for x in range(0, numCols):
            if scoresData[x] < min_confidence:
                continue
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
    return (rects, confidences)

# (rects, confidences) = decode_predictions(scores, geometry, min_confidence)
# boxes = non_max_suppression(np.array(rects), probs=confidences)

def detect_text(image, east_model_path, min_confidence=0.5, width=320, height=320):
    # Check if image is a string (path), then load it
    if isinstance(image, str):
        image = cv2.imread(image)
        if image is None:
            raise ValueError("Image not found at provided path.")
    orig = image.copy()
    (origH, origW) = image.shape[:2]
    rW = origW / float(width)
    rH = origH / float(height)
    image = cv2.resize(image, (width, height))
    (H, W) = image.shape[:2]

    # Define the two output layer names for the EAST detector model
    layerNames = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]
    net = cv2.dnn.readNet(east_model_path)
    # Construct a blob from the image and then perform a forward pass of the model
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H), (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)
    (rects, confidences) = decode_predictions(scores, geometry, min_confidence)
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    results = []
    for (startX, startY, endX, endY) in boxes:
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
        results.append((startX, startY, endX, endY))

    return results

def recognize_text(image, bounding_boxes):
    recognized_texts = []
    for box in bounding_boxes:
        x, y, w, h = box
        roi = image[y:y+h, x:x+w]
        text = pytesseract.image_to_string(roi)
        recognized_texts.append(text)
    return recognized_texts

In [ ]:
class ImageDescriptionPipeline:
    def __init__(self, east_model_path):
        # Load YOLOv8 (using YOLOv5 as a placeholder)
        self.yolov8_model = YOLO('yolov8n.pt')
        # Load GPT-2
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
        self.east_model_path = east_model_path

    def detect_objects(self, image):
        class_name = self.yolov8_model.names[int(class_id)]

        results = self.yolov8_model(image)
        detected_objects = []
        if hasattr(results, 'xyxy'):
            # Iterate over detections
            for detection in results.xyxy[0]:
                class_id = int(detection[5])
                confidence = float(detection[4])
                x_min, y_min, x_max, y_max = map(int, detection[:4])
                detected_objects.append({
                    'class_id': class_id,
                    'confidence': confidence,
                    'bbox': [x_min, y_min, x_max, y_max]
                })
        else:
            # Handle cases where the output is not as expected
            print("Unexpected output format from YOLO model")
        return detected_objects


    def detect_and_recognize_text(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image not found at provided path.")
        bounding_boxes = detect_text(image, self.east_model_path)
        recognized_texts = recognize_text(image, bounding_boxes)
        return recognized_texts

    def generate_description(self, objects, texts):
        # Generate description with GPT-2
        input_text = "Detected objects: " + ", ".join([str(obj['class_id']) for obj in objects])
        input_text += ". Detected texts: " + ", ".join(texts)
        inputs = self.tokenizer(input_text, return_tensors="pt")
        # Generate description using GPT-2
        outputs = self.gpt2_model.generate(inputs['input_ids'], max_length=525, num_return_sequences=1)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def describe_image(self, image_path):
        # Describe an image
        # image = cv2.imread(image_path)
        objects = self.detect_objects(image)
        texts = self.detect_and_recognize_text(image)
        description = self.generate_description(objects, texts)
        return description

# Example usage
east_model_path = '/content/frozen_east_text_detection.pb'
pipeline = ImageDescriptionPipeline(east_model_path)
image = '124.jpg'
description = pipeline.describe_image(image)
print(description)

UnboundLocalError: ignored

# Second Try

In [ ]:
# Function to decode predictions from EAST detector
def decode_predictions(scores, geometry, min_confidence):
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
    for y in range(numRows):
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]
        for x in range(numCols):
            if scoresData[x] < min_confidence:
                continue
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
    return (rects, confidences)

# Function to detect text in an image
def detect_text(image, east_model_path, min_confidence=0.5, width=320, height=320):
    if isinstance(image, str):
        image = cv2.imread(image)
        if image is None:
            raise ValueError("Image not found at provided path.")
    orig = image.copy()
    (origH, origW) = image.shape[:2]
    rW = origW / float(width)
    rH = origH / float(height)
    image = cv2.resize(image, (width, height))
    (H, W) = image.shape[:2]
    layerNames = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]
    net = cv2.dnn.readNet(east_model_path)
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H), (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)
    (rects, confidences) = decode_predictions(scores, geometry, min_confidence)
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    results = []
    for (startX, startY, endX, endY) in boxes:
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
        results.append((startX, startY, endX, endY))
    return results

def recognize_text(image, bounding_boxes):
    recognized_texts = set()  # Using a set to automatically avoid duplicates
    for box in bounding_boxes:
        x, y, w, h = box
        roi = image[y:y+h, x:x+w]
        text = pytesseract.image_to_string(roi).strip()
        if text:  # Only add non-empty text
            recognized_texts.add(text)
    return list(recognized_texts)

In [ ]:
import torch
import cv2
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from ultralytics import YOLO

class ImageDescriptionPipeline:
    def __init__(self, east_model_path, yolo_model_path):
        # Load YOLOv8
        self.yolov8_model = YOLO(yolo_model_path)
        # Load GPT-2
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
        self.east_model_path = east_model_path

    def detect_objects(self, image_path):
        # Load the image using OpenCV
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image not found at provided path: " + image_path)

        # Perform object detection using YOLO
        results = self.yolov8_model(image)

        # Process detection results
        detected_objects = []
        for result in results.pred:
            for det in result:
                x_min, y_min, x_max, y_max, confidence, class_id = det.tolist()
                class_name = self.yolov8_model.names[int(class_id)] if int(class_id) < len(self.yolov8_model.names) else "Unknown"

                detected_objects.append({
                    'class_id': int(class_id),
                    'class_name': class_name,
                    'confidence': float(confidence),
                    'bbox': [int(x_min), int(y_min), int(x_max), int(y_max)]
                })

        return detected_objects

    def detect_and_recognize_text(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image not found at provided path.")
        bounding_boxes = detect_text(image, self.east_model_path)
        recognized_texts = recognize_text(image, bounding_boxes)
        pass

    def generate_description(self, objects, texts):
        # Create a text description based on detected objects and recognized text
        object_descriptions = [f"{obj['class_name']} ({obj['confidence']:.2f})" for obj in objects]
        objects_text = ", ".join(object_descriptions) if object_descriptions else "No objects detected"

        texts_text = ", ".join(texts) if texts else "No text detected"

        combined_description = f"Detected objects: {objects_text}. Detected texts: {texts_text}"

        # Tokenize and generate description using GPT-2
        inputs = self.tokenizer(combined_description, return_tensors="pt")
        outputs = self.gpt2_model.generate(inputs['input_ids'], max_length=525, num_return_sequences=1)

        decoded_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        return decoded_output

    def describe_image(self, image_path):
        # Detect objects in the image
        objects = self.detect_objects(image_path)

        # Detect and recognize text in the image (Implement this part separately)

        # Placeholder for recognized texts, replace with your implementation
        texts = []

        # Generate the image description
        description = self.generate_description(objects, texts)
        return description

# Example usage
east_model_path = '/content/frozen_east_text_detection.pb'
yolo_model_path = 'yolov8n.pt'
pipeline = ImageDescriptionPipeline(east_model_path, yolo_model_path)
image_path = '124.jpg'
description = pipeline.describe_image(image_path)
print(description)

   # def describe_image(self, image_path):
  #      objects = self.detect_objects(image_path)
   #     texts = self.detect_and_recognize_text(image_path)
   #     description = self.generate_description(objects, texts)
   #     return description



0: 448x640 20 persons, 1 traffic light, 1 umbrella, 240.4ms
Speed: 7.0ms preprocess, 240.4ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)


AttributeError: ignored

In [ ]:
# Example usage
east_model_path = '/content/frozen_east_text_detection.pb'
yolo_model_path = 'yolov8n.pt'
pipeline = ImageDescriptionPipeline(east_model_path, yolo_model_path)
image_path = '124.jpg'

# Detect objects using YOLOv8
detection_results = pipeline.detect_objects(image_path)

# Generate description based on detection results
description = pipeline.generate_description(detection_results)
print(description)


image 1/1 /content/124.jpg: 448x640 11 persons, 3 backpacks, 1105.1ms
Speed: 13.8ms preprocess, 1105.1ms inference, 30.0ms postprocess per image at shape (1, 3, 448, 640)


AttributeError: ignored

In [ ]:
print(type(results))
print(results)

<class 'list'>
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58:

In [ ]:
def detect_objects(self, image):
    results = self.yolov8_model(image)
    detected_objects = []
    for result in results:
        # Adapt this loop based on actual structure of result
        for det in result:
            # Validate and parse det based on its structure
            if some_condition:  # Replace with actual condition
                class_id = int(det[some_index])  # Replace some_index with actual index
                class_name = self.yolov8_model.names[class_id]
                confidence = float(det[another_index])  # Replace another_index with actual index
                x_min, y_min, x_max, y_max = map(int, det[:4])  # Adjust slicing based on structure
                detected_objects.append({
                    'class_id': class_id,
                    'class_name': class_name,
                    'confidence': confidence,
                    'bbox': [x_min, y_min, x_max, y_max]
                })
            else:
                print("Unexpected detection format:", det)
    return detected_objects